In [1]:
import tensorflow as tf

2023-12-24 06:23:20.729262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [36]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from pytorch_transformers import XLNetModel, XLNetTokenizer
from transformers import XLNetForSequenceClassification
from pytorch_transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [3]:
test = pd.read_csv('Datasets/test_essays.csv')
sub = pd.read_csv('Datasets/sample_submission.csv')
org_train = pd.read_csv('Datasets/train_essays.csv')
train = pd.read_csv("Datasets/train_v2_drcat_02.csv", sep=',')

In [4]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

In [5]:
org_train.rename(columns = {'generated':'label'}, inplace=True)

In [6]:
train_data = pd.concat([train[['text','label']], org_train[['text','label']]])

In [15]:
sentences = train_data.text.values

In [17]:
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = train_data.label.values

In [20]:
sentences[0]

"Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.\n\nWhen people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.\n\nDriving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to reme

In [21]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 798011/798011 [00:01<00:00, 662620.73B/s]


Tokenize the first sentence:
['▁phones', '▁modern', '▁humans', '▁today', '▁are', '▁always', '▁on', '▁their', '▁phone', '.', '▁they', '▁are', '▁always', '▁on', '▁their', '▁phone', '▁more', '▁than', '▁5', '▁hours', '▁a', '▁day', '▁no', '▁stop', '▁', '.', 'all', '▁they', '▁do', '▁is', '▁text', '▁back', '▁and', '▁forward', '▁and', '▁just', '▁have', '▁group', '▁chat', 's', '▁on', '▁social', '▁media', '.', '▁they', '▁even', '▁do', '▁it', '▁while', '▁driving', '.', '▁they', '▁are', '▁some', '▁really', '▁bad', '▁consequences', '▁when', '▁stuff', '▁happens', '▁when', '▁it', '▁comes', '▁to', '▁a', '▁phone', '.', '▁some', '▁certain', '▁areas', '▁in', '▁the', '▁united', '▁states', '▁ban', '▁phones', '▁from', '▁class', '▁rooms', '▁just', '▁because', '▁of', '▁it', '.', '▁when', '▁people', '▁have', '▁phones', ',', '▁they', '▁know', '▁about', '▁certain', '▁apps', '▁that', '▁they', '▁have', '▁', '.', 'app', 's', '▁like', '▁face', 'book', '▁twitter', '▁in', 'sta', 'gram', '▁and', '▁snap', 'cha', 't', '.

In [22]:
MAX_LEN = 128

In [23]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [26]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [27]:

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


In [28]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=56, test_size=0.2)

In [29]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [33]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [37]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [38]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [39]:
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [47]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:  25%|██▌       | 1/4 [15:22<46:08, 922.73s/it]

Train loss: 0.043495506795796154


Epoch:  50%|█████     | 2/4 [30:45<30:45, 922.86s/it]

Train loss: 0.014145305058825318


Epoch:  75%|███████▌  | 3/4 [46:08<15:22, 922.85s/it]

Train loss: 0.00981349447274758


Epoch: 100%|██████████| 4/4 [1:01:31<00:00, 922.76s/it]

Train loss: 0.007508797742750801


In [48]:
torch.save(model.state_dict(), 'model_XLNET-baseline.ckpt')

In [49]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [50]:
print('Test Accuracy of the model on vla data is: {} %'.format(100 * correct / total))

Test Accuracy of the model on vla data is: 99.28648648648648 %


In [115]:
test = pd.read_csv('Datasets/test_essays.csv')

In [116]:
test

,id,prompt_id,text
0,0000aaaa,2,"Dress codes rulz! I mean, they're like, super ..."
1,1111bbbb,3,"In recent years, there has been a growing move..."
2,2222cccc,4,"Hey, so for this essay, I'm gonna talk about h..."


In [117]:
test_sentences = test.text.values

In [118]:
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]

In [119]:
test_tokenized_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
print ("Tokenize the first sentence:")
print (test_tokenized_texts[0])

Tokenize the first sentence:
['▁dress', '▁codes', '▁', 'r', 'ul', 'z', '!', '▁', 'i', '▁mean', ',', '▁they', "'", 're', '▁like', ',', '▁super', '▁important', '▁for', '▁creating', '▁a', '▁good', '▁learning', '▁environment', '.', '▁', 'i', '▁know', '▁some', '▁p', 'pl', '▁might', '▁think', '▁they', "'", 're', ',', '▁like', ',', '▁restrictive', '▁or', '▁whatever', ',', '▁but', '▁honestly', ',', '▁they', '▁help', '▁keep', '▁p', 'pl', '▁in', '▁line', '.', '▁first', '▁of', '▁all', ',', '▁dress', '▁codes', '▁promote', '▁equality', '.', '▁like', ',', '▁if', '▁everyone', "'", 's', '▁wearing', '▁the', '▁same', '▁thing', ',', '▁then', '▁no', '▁one', "'", 's', '▁gonna', '▁feel', '▁left', '▁out', '▁or', '▁judged', '.', '▁it', "'", 's', ',', '▁like', ',', '▁a', '▁way', '▁to', '▁eliminate', '▁bullying', '▁and', '▁stuff', '.', '▁and', '▁it', "'", 's', '▁not', '▁just', '▁about', '▁clothes', ',', '▁it', "'", 's', '▁about', ',', '▁like', ',', '▁respect', 'ing', '▁the', '▁school', '▁and', '▁the', '▁teacher

In [120]:
test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in test_tokenized_texts]

In [121]:
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [122]:
# Create attention masks
test_attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

In [123]:
test_inputs = torch.tensor(test_input_ids)
test_masks = torch.tensor(test_attention_masks)

In [124]:
test_data = TensorDataset(test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [125]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      

[0.9939180612564087, 0.995185911655426, 0.9943175911903381]

In [144]:
preds = torch.sigmoid(outputs[0])[:,1].tolist()

In [128]:
sub_data = {'id':test.id.values,'generated':preds}

In [131]:
submissions = pd.DataFrame(sub_data)

In [132]:
submissions.to_csv("Submissions.csv",index=False)